In [24]:
from dataclasses import dataclass, asdict
from typing import Dict, Any

from bg_atlasapi import BrainGlobeAtlas
from treelib import Tree
from numpy.random import randint
import pandas as pd

## What Data is in the Brainglobe Structures Tree, how do you get at it?

In [2]:
atlas = BrainGlobeAtlas("allen_mouse_100um")

allen mouse atlas (res. 100um)
From: http://www.brain-map.org (Wang et al 2020, https://doi.org/10.1016/j.cell.2020.04.007 )


In [5]:
tree = atlas.structures.tree

In [6]:
tree.all_nodes()[:10]

[Node(tag=root (997), identifier=997, data=None),
 Node(tag=grey (8), identifier=8, data=None),
 Node(tag=CH (567), identifier=567, data=None),
 Node(tag=CTX (688), identifier=688, data=None),
 Node(tag=CTXpl (695), identifier=695, data=None),
 Node(tag=Isocortex (315), identifier=315, data=None),
 Node(tag=FRP (184), identifier=184, data=None),
 Node(tag=FRP1 (68), identifier=68, data=None),
 Node(tag=FRP2/3 (667), identifier=667, data=None),
 Node(tag=FRP5 (526157192), identifier=526157192, data=None)]

In [7]:
tree.parent(8)

Node(tag=root (997), identifier=997, data=None)

In [10]:
atlas.structures[8]

{'acronym': 'grey', 'id': 8, 'name': 'Basic cell groups and regions', 'structure_id_path': [997, 8], 'rgb_triplet': [191, 218, 227], 'mesh_filename': WindowsPath('C:/Users/Nick/.brainglobe/allen_mouse_100um_v1.2/meshes/8.obj'), 'mesh': None}

## Let's Make a new, custom tree with nodes thet contain our own data structure

### BrainRegion Data Structure (will be attached to each node)

In [20]:
@dataclass
class BrainRegion:
    id: int
    name: str
    full_name: str
    depth: int
    cells: int
    total_cells: int = 0
        
    @property
    def repr(self) -> str:
        return f"{self.full_name} ({self.id}): {self.depth}|{self.cells}|{self.total_cells}"
    
    def to_dict(self) -> Dict[str, Any]:
        return asdict(self)
        

### Assemble and Populate new tree of BrainRegions from Brainglobe Atlas

In [33]:
def build_brainregion_tree_from_bgatlas(atlas):
    tree = atlas.structures.tree
    tree2 = Tree()
    for node in tree.all_nodes_itr():
        parent = tree.parent(node.identifier)
        data = atlas.structures.get(node.identifier)
        tree2.create_node(
            identifier=node.identifier, 
            parent=parent, 
            data=BrainRegion(
                id=int(node.identifier),
                name=data['acronym'] if data else node.tag.split(' (')[0],
                full_name = data['name'] if data else node.tag.split(' (')[0],
                depth=tree.depth(node),
                cells=randint(0, 3) if node.is_leaf() else 0  # Just assign a random amount of cells,
            )
        )
        
    for node in tree2.all_nodes_itr():
        subtree = tree2.subtree(node.identifier)
        cells = sum(n.data.cells for n in subtree.all_nodes_itr())
        node.data.total_cells = cells
        
    return tree2


tree = build_brainregion_tree_from_bgatlas(atlas)
root_node = 348
tree.show(root_node, data_property='repr')

Midbrain, behavioral state related (348): 4|0|17
├── Midbrain raphe nuclei (165): 5|0|13
│   ├── Interfascicular nucleus raphe (12): 6|0|0
│   ├── Interpeduncular nucleus (100): 6|0|9
│   │   ├── Interpeduncular nucleus, rostral (607344834): 7|2|2
│   │   ├── Interpeduncular nucleus, caudal (607344838): 7|0|0
│   │   ├── Interpeduncular nucleus, apical (607344842): 7|2|2
│   │   ├── Interpeduncular nucleus, lateral (607344846): 7|1|1
│   │   ├── Interpeduncular nucleus, intermediate (607344850): 7|1|1
│   │   ├── Interpeduncular nucleus, dorsomedial (607344854): 7|2|2
│   │   ├── Interpeduncular nucleus, dorsolateral (607344858): 7|1|1
│   │   └── Interpeduncular nucleus, rostrolateral (607344862): 7|0|0
│   ├── Rostral linear nucleus raphe (197): 6|2|2
│   ├── Central linear nucleus raphe (591): 6|2|2
│   └── Dorsal nucleus raphe (872): 6|0|0
├── Substantia nigra, compact part (374): 5|2|2
└── Pedunculopontine nucleus (1052): 5|2|2



## Convert to Pandas Dataframe of BrainRegions

In [37]:
df = pd.DataFrame([tree.get_node(node).data.to_dict() for node in tree.expand_tree()])
df.head(10)

,id,name,full_name,depth,cells,total_cells
0,997,root,root,0,0,657
1,8,grey,Basic cell groups and regions,1,0,586
2,343,BS,Brain stem,2,0,236
3,313,MB,Midbrain,3,0,55
4,323,MBmot,"Midbrain, motor related",4,0,26
5,35,III,Oculomotor nucleus,5,0,0
6,58,MT,Medial terminal nucleus of the accessory optic...,5,1,1
7,66,LT,Lateral terminal nucleus of the accessory opti...,5,0,0
8,75,DT,Dorsal terminal nucleus of the accessory optic...,5,2,2
9,115,IV,Trochlear nucleus,5,0,0


#### Query Dataframe by Region

In [40]:
df[df.depth==1]

,id,name,full_name,depth,cells,total_cells
1,8,grey,Basic cell groups and regions,1,0,586
724,73,VS,ventricular systems,1,0,4
733,1009,fiber tracts,fiber tracts,1,0,67


#### Query Dataframe by Name

In [41]:
df[df.name == 'IV']

,id,name,full_name,depth,cells,total_cells
9,115,IV,Trochlear nucleus,5,0,0


#### Query Dataframe by Number of cells

In [45]:
df[df.cells > 1].head()

,id,name,full_name,depth,cells,total_cells
8,75,DT,Dorsal terminal nucleus of the accessory optic...,5,2,2
13,246,RR,"Midbrain reticular nucleus, retrorubral area",5,2,2
19,381,SNr,"Substantia nigra, reticular part",5,2,2
20,616,CUN,Cuneiform nucleus,5,2,2
26,587,ND,Nucleus of Darkschewitsch,6,2,2
